In [ ]:
from itertools import combinations
from z3 import *
import time
import math
from utils import print_weekly_schedule

# Naive/Pairwise (NP)
def at_least_one_np(bool_vars):
    if not bool_vars: return BoolVal(False)
    return Or(bool_vars)

def at_most_one_np(bool_vars):
    if len(bool_vars) < 2: return []
    return [Not(And(bool_vars[i], bool_vars[j])) for i in range(len(bool_vars) - 1) for j in range(i+1, len(bool_vars))]

def exactly_one_np(bool_vars, name=''):
    return And(*at_most_one_np(bool_vars), at_least_one_np(bool_vars))
#--------------------------------------------------------------------------------------------------------------------------------------------------------

# Binary Encoding (BW)
def toBinary(num, length = None):
    num_bin = bin(num).split("b")[-1]
    if length:
        return "0"*(length - len(num_bin)) + num_bin
    return num_bin
    
def at_least_one_bw(bool_vars):
    return at_least_one_np(bool_vars)

def at_most_one_bw(bool_vars, name=''):
    constraints = []
    n = len(bool_vars)
    if n == 0: return BoolVal(True)
    
    m = math.ceil(math.log2(n))
    r = [Bool(f"r_{name}_{i}") for i in range(m)]
    binaries = [toBinary(idx, m) for idx in range(n)]
    
    for i in range(n):
        phi_parts = []
        for j in range(m):
            if binaries[i][j] == "1":
                phi_parts.append(r[j])
            else:
                phi_parts.append(Not(r[j]))
        constraints.append(Or(Not(bool_vars[i]), And(*phi_parts)))

    return And(constraints)

def exactly_one_bw(bool_vars, name=''):
    return And(at_least_one_bw(bool_vars), at_most_one_bw(bool_vars, name))
#--------------------------------------------------------------------------------------------------------------------------------------------------------

# Sequential Encoding (SEQ for k=1)
def at_least_one_seq(bool_vars):
    return at_least_one_np(bool_vars)

def at_most_one_seq(bool_vars, name=''):
    constraints = []
    n = len(bool_vars)
    if n == 0: return BoolVal(True)
    if n == 1: return BoolVal(True) 

    s = [Bool(f"s_{name}_{i}") for i in range(n - 1)]

    constraints.append(Or(Not(bool_vars[0]), s[0]))
    constraints.append(Or(Not(bool_vars[n-1]), Not(s[n-2])))
    
    for i in range(1, n - 1):
        constraints.append(Or(Not(bool_vars[i]), s[i]))
        constraints.append(Or(Not(bool_vars[i]), Not(s[i-1])))
        constraints.append(Or(Not(s[i-1]), s[i]))
    
    return And(constraints)

def exactly_one_seq(bool_vars, name=''):
    return And(at_least_one_seq(bool_vars), at_most_one_seq(bool_vars, name))
#--------------------------------------------------------------------------------------------------------------------------------------------------------

# General K-Encoding (NP - Direct Encoding)
def at_least_k_np(bool_vars, k, name = ""):
    return at_most_k_np([Not(var) for var in bool_vars], len(bool_vars)-k, name)

def at_most_k_np(bool_vars, k, name = ""):
    if k >= len(bool_vars): return BoolVal(True) 
    if k < 0: return BoolVal(False)
    return And([Or([Not(x) for x in X]) for X in combinations(bool_vars, k + 1)])

def exactly_k_np(bool_vars, k, name = ""):
    return And(at_most_k_np(bool_vars, k, name), at_least_k_np(bool_vars, k, name))


# General K-Encoding (SEQ - Sequential Counter Encoding)
def at_most_k_seq(bool_vars, k, name=''):
    constraints = []
    n = len(bool_vars)
    
    if n == 0: return BoolVal(True)
    if k == 0: return And([Not(v) for v in bool_vars])
    if k >= n: return BoolVal(True)

    s = [[Bool(f"s_{name}_{i}_{j}") for j in range(k)] for i in range(n)] 
    
    constraints.append(Or(Not(bool_vars[0]), s[0][0])) 
    for j in range(1, k):
        constraints.append(Not(s[0][j]))

    for i in range(1, n):
        constraints.append(Or(Not(s[i-1][0]), s[i][0]))
        constraints.append(Or(Not(bool_vars[i]), s[i][0]))

        for j in range(1, k):
            constraints.append(Or(Not(s[i-1][j]), s[i][j]))
            constraints.append(Or(Not(bool_vars[i]), Not(s[i-1][j-1]), s[i][j]))
        
        constraints.append(Or(Not(bool_vars[i]), Not(s[i-1][k-1])))

    return And(constraints)

def at_least_k_seq(bool_vars, k, name=''):
    return at_most_k_seq([Not(var) for var in bool_vars], len(bool_vars)-k, name + "_neg")

def exactly_k_seq(bool_vars, k, name=''):
    return And(at_most_k_seq(bool_vars, k, name), at_least_k_seq(bool_vars, k, name))
#--------------------------------------------------------------------------------------------------------------------------------------------------------

# --- Heule Encoding Approach (from the other group's code) ---
# This will be used specifically for exactly_one in the STS problem.
# The original `at_most_one_he` in your code was different.
global_most_counter = 0 # Renamed to avoid conflict with potential local vars

def heule_at_most_one(bool_vars):
    # This is the recursive Heule AMO used by the other group
    if len(bool_vars) <= 4: # Base case: use pairwise encoding
        return And([Not(And(pair[0], pair[1])) for pair in combinations(bool_vars, 2)])
    else:
        global global_most_counter
        global_most_counter += 1
        aux_var = Bool(f'y_amo_{global_most_counter}') # Using a distinct name for auxiliary vars

        # This recursive decomposition is the core of their Heule encoding
        # It splits into roughly 1/4 and 3/4, with an auxiliary variable
        return And(heule_at_most_one(bool_vars[:3] + [aux_var]), heule_at_most_one([Not(aux_var)] + bool_vars[3:]))

def heule_exactly_one(bool_vars, name=''):
    # Uses the Heule AMO and the simple at_least_one
    return And(heule_at_most_one(bool_vars), at_least_one_np(bool_vars)) # Using your at_least_one_np


# --- Model Creation Function (adapted from the other group's `create_model`) ---
# This function will set up the core STS constraints and variables.
# It returns the solver, games_vars, home_counts, away_counts, diff_values, total_objective
# to be used in the solve method.
def create_sts_model(n, exactly_one_encoding, at_most_k_encoding):
    NUM_TEAMS = n
    NUM_WEEKS = NUM_TEAMS - 1
    NUM_PERIODS_PER_WEEK = n // 2
    
    if NUM_TEAMS % 2 != 0:
        raise ValueError(f"Error: The number of teams (n={NUM_TEAMS}) must be an even number.")

    solver_sts = Solver()

    # --- Decision Variables ---
    games_vars = {}
    for i in range(NUM_TEAMS):
        for j in range(NUM_TEAMS):
            if i != j:
                for w in range(NUM_WEEKS):
                    for p in range(NUM_PERIODS_PER_WEEK):
                        games_vars[(i, j, w, p)] = Bool(f'g_{i+1}_{j+1}_{w+1}_{p+1}')
                        
    # --- Base Constraints ---
    # 1. Every pair plays exactly once (home or away)
    for i in range(NUM_TEAMS):
        for j in range(i+1, NUM_TEAMS):
            i_j_all_possible_matches = []
            for w in range(NUM_WEEKS):
                for p in range(NUM_PERIODS_PER_WEEK):
                    i_j_all_possible_matches.append(games_vars[(i, j, w, p)])
                    i_j_all_possible_matches.append(games_vars[(j, i, w, p)])
            solver_sts.add(exactly_one_encoding(i_j_all_possible_matches, f"pair_once_{i}_{j}"))
    
            
    # 2. Every team plays exactly once a week
    for i in range(NUM_TEAMS):
        for w in range(NUM_WEEKS):
            once_week_game = []
            for j in range(NUM_TEAMS):
                if i == j: continue
                for p in range(NUM_PERIODS_PER_WEEK):
                    once_week_game.append(games_vars[(i,j,w,p)])
                    once_week_game.append(games_vars[(j,i,w,p)])
            solver_sts.add(exactly_one_encoding(once_week_game, f"team_once_week_{i}_{w}"))
            
    # 3. Every team plays at most twice in the same period over the tournament
    for i in range(NUM_TEAMS):
        for p in range(NUM_PERIODS_PER_WEEK):
            num_games_period = []
            for j in range(NUM_TEAMS):
                if i == j: continue
                for w in range(NUM_WEEKS):
                    num_games_period.append(games_vars[(i,j,w,p)])
                    num_games_period.append(games_vars[(j,i,w,p)])
            solver_sts.add(at_most_k_encoding(num_games_period, 2, f"team_at_most_2_period_{i}_{p}"))
            
    # 4. Exactly one game in each period of every week
    for w in range(NUM_WEEKS):
        for p in range(NUM_PERIODS_PER_WEEK):
            game_in_period = []
            for i in range(NUM_TEAMS):
                for j in range(i+1, NUM_TEAMS):
                    game_in_period.append(games_vars[(i,j,w,p)])
                    game_in_period.append(games_vars[(j,i,w,p)])
            solver_sts.add(exactly_one_encoding(game_in_period, f"slot_one_game_{w}_{p}"))
            
    # Symmetry Breaking 
    
    # 1. impose team that team one plays against team 2 in the first period during the first week
    if NUM_TEAMS >= 2 and NUM_WEEKS >= 1 and NUM_PERIODS_PER_WEEK >= 1:
        solver_sts.add(games_vars[(0, 1, 0, 0)])

    # --- Optimization Variables ---
    home_counts = [Int(f'H_{i+1}') for i in range(NUM_TEAMS)]
    away_counts = [Int(f'A_{i+1}') for i in range(NUM_TEAMS)]
    diff_values = [Int(f'D_{i+1}') for i in range(NUM_TEAMS)] 

    # Constraints for home/away counts, using Z3's If and Sum for aggregation
    for i in range(NUM_TEAMS):
        home_games_for_team_i_bools = [games_vars[(i, j, w, p)] 
                                        for j in range(NUM_TEAMS) if i != j 
                                        for w in range(NUM_WEEKS) 
                                        for p in range(NUM_PERIODS_PER_WEEK)]
        solver_sts.add(home_counts[i] == Sum([If(v, 1, 0) for v in home_games_for_team_i_bools]))

        away_games_for_team_i_bools = [games_vars[(j, i, w, p)] 
                                        for j in range(NUM_TEAMS) if i != j 
                                        for w in range(NUM_WEEKS) 
                                        for p in range(NUM_PERIODS_PER_WEEK)]
        solver_sts.add(away_counts[i] == Sum([If(v, 1, 0) for v in away_games_for_team_i_bools]))

        solver_sts.add(home_counts[i] + away_counts[i] == NUM_WEEKS) 
        
        # Vincola diff_values[i] ad essere esattamente il valore assoluto
        solver_sts.add(diff_values[i] == If(home_counts[i] >= away_counts[i],
                                       home_counts[i] - away_counts[i],
                                       away_counts[i] - home_counts[i]))

    # Total difference (objective function)
    total_objective = Sum(diff_values)
    
    # --- ADDING THE LOWER BOUND CONSTRAINT HERE ---
    # Since n is always even, NUM_WEEKS (n-1) is always odd.
    # The minimum difference |H-A| for a single team is 1.
    # So, the sum of differences for all N_TEAMS is at least N_TEAMS * 1.
    lower_bound_total_diff = NUM_TEAMS # This is 'n' in your problem context
    print(f"  Adding lower bound constraint: total_objective >= {lower_bound_total_diff}")
    solver_sts.add(total_objective >= lower_bound_total_diff)

    return solver_sts, games_vars, home_counts, away_counts, diff_values, total_objective


# --- Unified Model Class (adapted from the other group's `Unified_HeuleEnc_Model`) ---
class STS_Optimized_Model:
    def __init__(self, n, exactly_one_encoding, at_most_k_encoding):
        self.n = n
        self.NUM_TEAMS = n
        self.NUM_WEEKS = self.NUM_TEAMS - 1
        self.NUM_PERIODS_PER_WEEK = self.n // 2
        self.exactly_one_encoding = exactly_one_encoding
        self.at_most_k_encoding = at_most_k_encoding
        
        # in this way i can change the encodings as i prefer
        self.solver, self.games_vars, self.home_counts, self.away_counts, self.diff_values, self.total_objective = create_sts_model(n, self.exactly_one_encoding, self.at_most_k_encoding)

    def solve(self, timeout_seconds, random_seed=None):
        set_option("sat.local_search", True) # As in their code
        if random_seed is not None:
            self.solver.set("random_seed", random_seed)
            
        #self.solver.set("phase_selection", 0) # 0: Preferenza per polarità binarie. Prova anche 1 o 2.

        #self.solver.set("restart_factor", 1.2)
        #self.solver.set("restart_strategy", 1) # 0: GEOMETRIC, 1: LUBY.

        # Init for iterative optimization
        optimal_objective_value = None
        best_solution_schedule = []
        init_time = time.time()
        timeout_timestamp = init_time + timeout_seconds

        print(f"\n--- Starting Optimization for n={self.n} ---")
        print(f"  Using exactly_one: {self.exactly_one_encoding.__name__}, at_most_k: {self.at_most_k_encoding.__name__}")

        while True:
            remaining_time = math.floor(timeout_timestamp - time.time())
            if remaining_time <= 0:
                print("  Overall timeout reached. Breaking optimization loop.")
                break # Overall timeout for the solve method

            self.solver.set("timeout", remaining_time * 1000) # Timeout for current check call in milliseconds
            self.solver.push() # Save state for back-tracking if current objective fails

            # Add constraint to find a strictly better solution
            if optimal_objective_value is not None:
                self.solver.add(self.total_objective < optimal_objective_value)
                print(f"  Searching for solution with total difference < {optimal_objective_value} (Remaining overall time: {remaining_time}s)...")
            else:
                print(f"  Searching for initial solution (Remaining overall time: {remaining_time}s)...")

            status = self.solver.check()
            current_elapsed_time = time.time() - init_time
            
            print(f"  Solver iteration returned: {status} (Elapsed: {current_elapsed_time:.2f}s)")

            if status == sat:
                model = self.solver.model()
                current_objective = model.evaluate(self.total_objective).as_long()
                
                print(f"    Found solution with total difference = {current_objective}")
                
                if optimal_objective_value is None or current_objective < optimal_objective_value:
                    optimal_objective_value = current_objective
                    print(f"    New best total difference found: {optimal_objective_value}")

                    best_solution_schedule = []
                    for (i, j, w, p), var in self.games_vars.items():
                        if model.evaluate(var):
                            best_solution_schedule.append((i+1, j+1, w+1, p+1))
# --- Model Creation Function (adapted from the other group's `create_model`) ---
# This function will set up the core STS constraints and variables.
# It returns the solver, games_vars, home_counts, away_counts, diff_values, total_objective
# to be used in the solve method.
def create_sts_model(n, exactly_one_encoding, at_most_k_encoding):
    NUM_TEAMS = n
    NUM_WEEKS = NUM_TEAMS - 1
    NUM_PERIODS_PER_WEEK = n // 2
    
    if NUM_TEAMS % 2 != 0:
        raise ValueError(f"Error: The number of teams (n={NUM_TEAMS}) must be an even number.")

    solver_sts = Solver()

    # --- Decision Variables ---
    games_vars = {}
    for i in range(NUM_TEAMS):
        for j in range(NUM_TEAMS):
            if i != j:
                for w in range(NUM_WEEKS):
                    for p in range(NUM_PERIODS_PER_WEEK):
                        games_vars[(i, j, w, p)] = Bool(f'g_{i+1}_{j+1}_{w+1}_{p+1}')
                        
    # --- Base Constraints ---
    # 1. Every pair plays exactly once (home or away)
    for i in range(NUM_TEAMS):
        for j in range(i+1, NUM_TEAMS):
            i_j_all_possible_matches = []
            for w in range(NUM_WEEKS):
                for p in range(NUM_PERIODS_PER_WEEK):
                    i_j_all_possible_matches.append(games_vars[(i, j, w, p)])
                    i_j_all_possible_matches.append(games_vars[(j, i, w, p)])
            solver_sts.add(exactly_one_encoding(i_j_all_possible_matches, f"pair_once_{i}_{j}"))
    
            
    # 2. Every team plays exactly once a week
    for i in range(NUM_TEAMS):
        for w in range(NUM_WEEKS):
            once_week_game = []
            for j in range(NUM_TEAMS):
                if i == j: continue
                for p in range(NUM_PERIODS_PER_WEEK):
                    once_week_game.append(games_vars[(i,j,w,p)])
                    once_week_game.append(games_vars[(j,i,w,p)])
            solver_sts.add(exactly_one_encoding(once_week_game, f"team_once_week_{i}_{w}"))
            
    # 3. Every team plays at most twice in the same period over the tournament
    for i in range(NUM_TEAMS):
        for p in range(NUM_PERIODS_PER_WEEK):
            num_games_period = []
            for j in range(NUM_TEAMS):
                if i == j: continue
                for w in range(NUM_WEEKS):
                    num_games_period.append(games_vars[(i,j,w,p)])
                    num_games_period.append(games_vars[(j,i,w,p)])
            solver_sts.add(at_most_k_encoding(num_games_period, 2, f"team_at_most_2_period_{i}_{p}"))
            
    # 4. Exactly one game in each period of every week
    for w in range(NUM_WEEKS):
        for p in range(NUM_PERIODS_PER_WEEK):
            game_in_period = []
            for i in range(NUM_TEAMS):
                for j in range(i+1, NUM_TEAMS):
                    game_in_period.append(games_vars[(i,j,w,p)])
                    game_in_period.append(games_vars[(j,i,w,p)])
            solver_sts.add(exactly_one_encoding(game_in_period, f"slot_one_game_{w}_{p}"))
            
    # Symmetry Breaking 
    
    # 1. impose team that team one plays against team 2 in the first period during the first week
    if NUM_TEAMS >= 2 and NUM_WEEKS >= 1 and NUM_PERIODS_PER_WEEK >= 1:
        solver_sts.add(games_vars[(0, 1, 0, 0)])

    # --- Optimization Variables ---
    home_counts = [Int(f'H_{i+1}') for i in range(NUM_TEAMS)]
    away_counts = [Int(f'A_{i+1}') for i in range(NUM_TEAMS)]
    diff_values = [Int(f'D_{i+1}') for i in range(NUM_TEAMS)] 

    # Constraints for home/away counts, using Z3's If and Sum for aggregation
    for i in range(NUM_TEAMS):
        home_games_for_team_i_bools = [games_vars[(i, j, w, p)] 
                                        for j in range(NUM_TEAMS) if i != j 
                                        for w in range(NUM_WEEKS) 
                                        for p in range(NUM_PERIODS_PER_WEEK)]
        solver_sts.add(home_counts[i] == Sum([If(v, 1, 0) for v in home_games_for_team_i_bools]))

        away_games_for_team_i_bools = [games_vars[(j, i, w, p)] 
                                        for j in range(NUM_TEAMS) if i != j 
                                        for w in range(NUM_WEEKS) 
                                        for p in range(NUM_PERIODS_PER_WEEK)]
        solver_sts.add(away_counts[i] == Sum([If(v, 1, 0) for v in away_games_for_team_i_bools]))

        solver_sts.add(home_counts[i] + away_counts[i] == NUM_WEEKS) 
        
        # Vincola diff_values[i] ad essere esattamente il valore assoluto
        solver_sts.add(diff_values[i] == If(home_counts[i] >= away_counts[i],
                                       home_counts[i] - away_counts[i],
                                       away_counts[i] - home_counts[i]))

    # Total difference (objective function)
    total_objective = Sum(diff_values)
    
    # --- ADDING THE LOWER BOUND CONSTRAINT HERE ---
    # Since n is always even, NUM_WEEKS (n-1) is always odd.
    # The minimum difference |H-A| for a single team is 1.
    # So, the sum of differences for all N_TEAMS is at least N_TEAMS * 1.
    lower_bound_total_diff = NUM_TEAMS # This is 'n' in your problem context
    print(f"  Adding lower bound constraint: total_objective >= {lower_bound_total_diff}")
    solver_sts.add(total_objective >= lower_bound_total_diff)

    return solver_sts, games_vars, home_counts, away_counts, diff_values, total_objective


# --- Unified Model Class (adapted from the other group's `Unified_HeuleEnc_Model`) ---
class STS_Optimized_Model:
    def __init__(self, n, exactly_one_encoding, at_most_k_encoding):
        self.n = n
        self.NUM_TEAMS = n
        self.NUM_WEEKS = self.NUM_TEAMS - 1
        self.NUM_PERIODS_PER_WEEK = self.n // 2
        self.exactly_one_encoding = exactly_one_encoding
        self.at_most_k_encoding = at_most_k_encoding
        
        # in this way i can change the encodings as i prefer
        self.solver, self.games_vars, self.home_counts, self.away_counts, self.diff_values, self.total_objective = create_sts_model(n, self.exactly_one_encoding, self.at_most_k_encoding)

    def solve(self, timeout_seconds, random_seed=None):
        set_option("sat.local_search", True) # As in their code
        if random_seed is not None:
            self.solver.set("random_seed", random_seed)
            
        #self.solver.set("phase_selection", 0) # 0: Preferenza per polarità binarie. Prova anche 1 o 2.

        #self.solver.set("restart_factor", 1.2)
        #self.solver.set("restart_strategy", 1) # 0: GEOMETRIC, 1: LUBY.

        # Init for iterative optimization
        optimal_objective_value = None
        best_solution_schedule = []
        init_time = time.time()
        timeout_timestamp = init_time + timeout_seconds

        print(f"\n--- Starting Optimization for n={self.n} ---")
        print(f"  Using exactly_one: {self.exactly_one_encoding.__name__}, at_most_k: {self.at_most_k_encoding.__name__}")

        while True:
            remaining_time = math.floor(timeout_timestamp - time.time())
            if remaining_time <= 0:
                print("  Overall timeout reached. Breaking optimization loop.")
                break # Overall timeout for the solve method

            self.solver.set("timeout", remaining_time * 1000) # Timeout for current check call in milliseconds
            self.solver.push() # Save state for back-tracking if current objective fails

            # Add constraint to find a strictly better solution
            if optimal_objective_value is not None:
                self.solver.add(self.total_objective < optimal_objective_value)
                print(f"  Searching for solution with total difference < {optimal_objective_value} (Remaining overall time: {remaining_time}s)...")
            else:
                print(f"  Searching for initial solution (Remaining overall time: {remaining_time}s)...")

            status = self.solver.check()
            current_elapsed_time = time.time() - init_time
            
            print(f"  Solver iteration returned: {status} (Elapsed: {current_elapsed_time:.2f}s)")

            if status == sat:
                model = self.solver.model()
                current_objective = model.evaluate(self.total_objective).as_long()
                
                print(f"    Found solution with total difference = {current_objective}")
                
                if optimal_objective_value is None or current_objective < optimal_objective_value:
                    optimal_objective_value = current_objective
                    print(f"    New best total difference found: {optimal_objective_value}")

                    best_solution_schedule = []
                    for (i, j, w, p), var in self.games_vars.items():
                        if model.evaluate(var):
                            best_solution_schedule.append((i+1, j+1, w+1, p+1))
# --- Model Creation Function (adapted from the other group's `create_model`) ---
# This function will set up the core STS constraints and variables.
# It returns the solver, games_vars, home_counts, away_counts, diff_values, total_objective
# to be used in the solve method.
def create_sts_model(n, exactly_one_encoding, at_most_k_encoding):
    NUM_TEAMS = n
    NUM_WEEKS = NUM_TEAMS - 1
    NUM_PERIODS_PER_WEEK = n // 2
    
    if NUM_TEAMS % 2 != 0:
        raise ValueError(f"Error: The number of teams (n={NUM_TEAMS}) must be an even number.")

    solver_sts = Solver()

    # --- Decision Variables ---
    games_vars = {}
    for i in range(NUM_TEAMS):
        for j in range(NUM_TEAMS):
            if i != j:
                for w in range(NUM_WEEKS):
                    for p in range(NUM_PERIODS_PER_WEEK):
                        games_vars[(i, j, w, p)] = Bool(f'g_{i+1}_{j+1}_{w+1}_{p+1}')
                        
    # --- Base Constraints ---
    # 1. Every pair plays exactly once (home or away)
    for i in range(NUM_TEAMS):
        for j in range(i+1, NUM_TEAMS):
            i_j_all_possible_matches = []
            for w in range(NUM_WEEKS):
                for p in range(NUM_PERIODS_PER_WEEK):
                    i_j_all_possible_matches.append(games_vars[(i, j, w, p)])
                    i_j_all_possible_matches.append(games_vars[(j, i, w, p)])
            solver_sts.add(exactly_one_encoding(i_j_all_possible_matches, f"pair_once_{i}_{j}"))
    
            
    # 2. Every team plays exactly once a week
    for i in range(NUM_TEAMS):
        for w in range(NUM_WEEKS):
            once_week_game = []
            for j in range(NUM_TEAMS):
                if i == j: continue
                for p in range(NUM_PERIODS_PER_WEEK):
                    once_week_game.append(games_vars[(i,j,w,p)])
                    once_week_game.append(games_vars[(j,i,w,p)])
            solver_sts.add(exactly_one_encoding(once_week_game, f"team_once_week_{i}_{w}"))
            
    # 3. Every team plays at most twice in the same period over the tournament
    for i in range(NUM_TEAMS):
        for p in range(NUM_PERIODS_PER_WEEK):
            num_games_period = []
            for j in range(NUM_TEAMS):
                if i == j: continue
                for w in range(NUM_WEEKS):
                    num_games_period.append(games_vars[(i,j,w,p)])
                    num_games_period.append(games_vars[(j,i,w,p)])
            solver_sts.add(at_most_k_encoding(num_games_period, 2, f"team_at_most_2_period_{i}_{p}"))
            
    # 4. Exactly one game in each period of every week
    for w in range(NUM_WEEKS):
        for p in range(NUM_PERIODS_PER_WEEK):
            game_in_period = []
            for i in range(NUM_TEAMS):
                for j in range(i+1, NUM_TEAMS):
                    game_in_period.append(games_vars[(i,j,w,p)])
                    game_in_period.append(games_vars[(j,i,w,p)])
            solver_sts.add(exactly_one_encoding(game_in_period, f"slot_one_game_{w}_{p}"))
            
    # Symmetry Breaking 
    
    # 1. impose team that team one plays against team 2 in the first period during the first week
    if NUM_TEAMS >= 2 and NUM_WEEKS >= 1 and NUM_PERIODS_PER_WEEK >= 1:
        solver_sts.add(games_vars[(0, 1, 0, 0)])

    # --- Optimization Variables ---
    home_counts = [Int(f'H_{i+1}') for i in range(NUM_TEAMS)]
    away_counts = [Int(f'A_{i+1}') for i in range(NUM_TEAMS)]
    diff_values = [Int(f'D_{i+1}') for i in range(NUM_TEAMS)] 

    # Constraints for home/away counts, using Z3's If and Sum for aggregation
    for i in range(NUM_TEAMS):
        home_games_for_team_i_bools = [games_vars[(i, j, w, p)] 
                                        for j in range(NUM_TEAMS) if i != j 
                                        for w in range(NUM_WEEKS) 
                                        for p in range(NUM_PERIODS_PER_WEEK)]
        solver_sts.add(home_counts[i] == Sum([If(v, 1, 0) for v in home_games_for_team_i_bools]))

        away_games_for_team_i_bools = [games_vars[(j, i, w, p)] 
                                        for j in range(NUM_TEAMS) if i != j 
                                        for w in range(NUM_WEEKS) 
                                        for p in range(NUM_PERIODS_PER_WEEK)]
        solver_sts.add(away_counts[i] == Sum([If(v, 1, 0) for v in away_games_for_team_i_bools]))

        solver_sts.add(home_counts[i] + away_counts[i] == NUM_WEEKS) 
        
        # Vincola diff_values[i] ad essere esattamente il valore assoluto
        solver_sts.add(diff_values[i] == If(home_counts[i] >= away_counts[i],
                                       home_counts[i] - away_counts[i],
                                       away_counts[i] - home_counts[i]))

    # Total difference (objective function)
    total_objective = Sum(diff_values)
    
    # --- ADDING THE LOWER BOUND CONSTRAINT HERE ---
    # Since n is always even, NUM_WEEKS (n-1) is always odd.
    # The minimum difference |H-A| for a single team is 1.
    # So, the sum of differences for all N_TEAMS is at least N_TEAMS * 1.
    lower_bound_total_diff = NUM_TEAMS # This is 'n' in your problem context
    print(f"  Adding lower bound constraint: total_objective >= {lower_bound_total_diff}")
    solver_sts.add(total_objective >= lower_bound_total_diff)

    return solver_sts, games_vars, home_counts, away_counts, diff_values, total_objective


# --- Unified Model Class (adapted from the other group's `Unified_HeuleEnc_Model`) ---
class STS_Optimized_Model:
    def __init__(self, n, exactly_one_encoding, at_most_k_encoding):
        self.n = n
        self.NUM_TEAMS = n
        self.NUM_WEEKS = self.NUM_TEAMS - 1
        self.NUM_PERIODS_PER_WEEK = self.n // 2
        self.exactly_one_encoding = exactly_one_encoding
        self.at_most_k_encoding = at_most_k_encoding
        
        # in this way i can change the encodings as i prefer
        self.solver, self.games_vars, self.home_counts, self.away_counts, self.diff_values, self.total_objective = create_sts_model(n, self.exactly_one_encoding, self.at_most_k_encoding)

    def solve(self, timeout_seconds, random_seed=None):
        set_option("sat.local_search", True) # As in their code
        if random_seed is not None:
            self.solver.set("random_seed", random_seed)
            
        #self.solver.set("phase_selection", 0) # 0: Preferenza per polarità binarie. Prova anche 1 o 2.

        #self.solver.set("restart_factor", 1.2)
        #self.solver.set("restart_strategy", 1) # 0: GEOMETRIC, 1: LUBY.

        # Init for iterative optimization
        optimal_objective_value = None
        best_solution_schedule = []
        init_time = time.time()
        timeout_timestamp = init_time + timeout_seconds

        print(f"\n--- Starting Optimization for n={self.n} ---")
        print(f"  Using exactly_one: {self.exactly_one_encoding.__name__}, at_most_k: {self.at_most_k_encoding.__name__}")

        while True:
            remaining_time = math.floor(timeout_timestamp - time.time())
            if remaining_time <= 0:
                print("  Overall timeout reached. Breaking optimization loop.")
                break # Overall timeout for the solve method

            self.solver.set("timeout", remaining_time * 1000) # Timeout for current check call in milliseconds
            self.solver.push() # Save state for back-tracking if current objective fails

            # Add constraint to find a strictly better solution
            if optimal_objective_value is not None:
                self.solver.add(self.total_objective < optimal_objective_value)
                print(f"  Searching for solution with total difference < {optimal_objective_value} (Remaining overall time: {remaining_time}s)...")
            else:
                print(f"  Searching for initial solution (Remaining overall time: {remaining_time}s)...")

            status = self.solver.check()
            current_elapsed_time = time.time() - init_time
            
            print(f"  Solver iteration returned: {status} (Elapsed: {current_elapsed_time:.2f}s)")

            if status == sat:
                model = self.solver.model()
                current_objective = model.evaluate(self.total_objective).as_long()
                
                print(f"    Found solution with total difference = {current_objective}")
                
                if optimal_objective_value is None or current_objective < optimal_objective_value:
                    optimal_objective_value = current_objective
                    print(f"    New best total difference found: {optimal_objective_value}")

                    best_solution_schedule = []
                    for (i, j, w, p), var in self.games_vars.items():
                        if model.evaluate(var):
                            best_solution_schedule.append((i+1, j+1, w+1, p+1))
# --- Model Creation Function (adapted from the other group's `create_model`) ---
# This function will set up the core STS constraints and variables.
# It returns the solver, games_vars, home_counts, away_counts, diff_values, total_objective
# to be used in the solve method.
def create_sts_model(n, exactly_one_encoding, at_most_k_encoding):
    NUM_TEAMS = n
    NUM_WEEKS = NUM_TEAMS - 1
    NUM_PERIODS_PER_WEEK = n // 2
    
    if NUM_TEAMS % 2 != 0:
        raise ValueError(f"Error: The number of teams (n={NUM_TEAMS}) must be an even number.")

    solver_sts = Solver()

    # --- Decision Variables ---
    games_vars = {}
    for i in range(NUM_TEAMS):
        for j in range(NUM_TEAMS):
            if i != j:
                for w in range(NUM_WEEKS):
                    for p in range(NUM_PERIODS_PER_WEEK):
                        games_vars[(i, j, w, p)] = Bool(f'g_{i+1}_{j+1}_{w+1}_{p+1}')
                        
    # --- Base Constraints ---
    # 1. Every pair plays exactly once (home or away)
    for i in range(NUM_TEAMS):
        for j in range(i+1, NUM_TEAMS):
            i_j_all_possible_matches = []
            for w in range(NUM_WEEKS):
                for p in range(NUM_PERIODS_PER_WEEK):
                    i_j_all_possible_matches.append(games_vars[(i, j, w, p)])
                    i_j_all_possible_matches.append(games_vars[(j, i, w, p)])
            solver_sts.add(exactly_one_encoding(i_j_all_possible_matches, f"pair_once_{i}_{j}"))
    
            
    # 2. Every team plays exactly once a week
    for i in range(NUM_TEAMS):
        for w in range(NUM_WEEKS):
            once_week_game = []
            for j in range(NUM_TEAMS):
                if i == j: continue
                for p in range(NUM_PERIODS_PER_WEEK):
                    once_week_game.append(games_vars[(i,j,w,p)])
                    once_week_game.append(games_vars[(j,i,w,p)])
            solver_sts.add(exactly_one_encoding(once_week_game, f"team_once_week_{i}_{w}"))
            
    # 3. Every team plays at most twice in the same period over the tournament
    for i in range(NUM_TEAMS):
        for p in range(NUM_PERIODS_PER_WEEK):
            num_games_period = []
            for j in range(NUM_TEAMS):
                if i == j: continue
                for w in range(NUM_WEEKS):
                    num_games_period.append(games_vars[(i,j,w,p)])
                    num_games_period.append(games_vars[(j,i,w,p)])
            solver_sts.add(at_most_k_encoding(num_games_period, 2, f"team_at_most_2_period_{i}_{p}"))
            
    # 4. Exactly one game in each period of every week
    for w in range(NUM_WEEKS):
        for p in range(NUM_PERIODS_PER_WEEK):
            game_in_period = []
            for i in range(NUM_TEAMS):
                for j in range(i+1, NUM_TEAMS):
                    game_in_period.append(games_vars[(i,j,w,p)])
                    game_in_period.append(games_vars[(j,i,w,p)])
            solver_sts.add(exactly_one_encoding(game_in_period, f"slot_one_game_{w}_{p}"))
            
    # Symmetry Breaking 
    
    # 1. impose team that team one plays against team 2 in the first period during the first week
    if NUM_TEAMS >= 2 and NUM_WEEKS >= 1 and NUM_PERIODS_PER_WEEK >= 1:
        solver_sts.add(games_vars[(0, 1, 0, 0)])

    # --- Optimization Variables ---
    home_counts = [Int(f'H_{i+1}') for i in range(NUM_TEAMS)]
    away_counts = [Int(f'A_{i+1}') for i in range(NUM_TEAMS)]
    diff_values = [Int(f'D_{i+1}') for i in range(NUM_TEAMS)] 

    # Constraints for home/away counts, using Z3's If and Sum for aggregation
    for i in range(NUM_TEAMS):
        home_games_for_team_i_bools = [games_vars[(i, j, w, p)] 
                                        for j in range(NUM_TEAMS) if i != j 
                                        for w in range(NUM_WEEKS) 
                                        for p in range(NUM_PERIODS_PER_WEEK)]
        solver_sts.add(home_counts[i] == Sum([If(v, 1, 0) for v in home_games_for_team_i_bools]))

        away_games_for_team_i_bools = [games_vars[(j, i, w, p)] 
                                        for j in range(NUM_TEAMS) if i != j 
                                        for w in range(NUM_WEEKS) 
                                        for p in range(NUM_PERIODS_PER_WEEK)]
        solver_sts.add(away_counts[i] == Sum([If(v, 1, 0) for v in away_games_for_team_i_bools]))

        solver_sts.add(home_counts[i] + away_counts[i] == NUM_WEEKS) 
        
        # Vincola diff_values[i] ad essere esattamente il valore assoluto
        solver_sts.add(diff_values[i] == If(home_counts[i] >= away_counts[i],
                                       home_counts[i] - away_counts[i],
                                       away_counts[i] - home_counts[i]))

    # Total difference (objective function)
    total_objective = Sum(diff_values)
    
    # --- ADDING THE LOWER BOUND CONSTRAINT HERE ---
    # Since n is always even, NUM_WEEKS (n-1) is always odd.
    # The minimum difference |H-A| for a single team is 1.
    # So, the sum of differences for all N_TEAMS is at least N_TEAMS * 1.
    lower_bound_total_diff = NUM_TEAMS # This is 'n' in your problem context
    print(f"  Adding lower bound constraint: total_objective >= {lower_bound_total_diff}")
    solver_sts.add(total_objective >= lower_bound_total_diff)

    return solver_sts, games_vars, home_counts, away_counts, diff_values, total_objective


# --- Unified Model Class (adapted from the other group's `Unified_HeuleEnc_Model`) ---
class STS_Optimized_Model:
    def __init__(self, n, exactly_one_encoding, at_most_k_encoding):
        self.n = n
        self.NUM_TEAMS = n
        self.NUM_WEEKS = self.NUM_TEAMS - 1
        self.NUM_PERIODS_PER_WEEK = self.n // 2
        self.exactly_one_encoding = exactly_one_encoding
        self.at_most_k_encoding = at_most_k_encoding
        
        # in this way i can change the encodings as i prefer
        self.solver, self.games_vars, self.home_counts, self.away_counts, self.diff_values, self.total_objective = create_sts_model(n, self.exactly_one_encoding, self.at_most_k_encoding)

    def solve(self, timeout_seconds, random_seed=None):
        set_option("sat.local_search", True) # As in their code
        if random_seed is not None:
            self.solver.set("random_seed", random_seed)
            
        #self.solver.set("phase_selection", 0) # 0: Preferenza per polarità binarie. Prova anche 1 o 2.

        #self.solver.set("restart_factor", 1.2)
        #self.solver.set("restart_strategy", 1) # 0: GEOMETRIC, 1: LUBY.

        # Init for iterative optimization
        optimal_objective_value = None
        best_solution_schedule = []
        init_time = time.time()
        timeout_timestamp = init_time + timeout_seconds

        print(f"\n--- Starting Optimization for n={self.n} ---")
        print(f"  Using exactly_one: {self.exactly_one_encoding.__name__}, at_most_k: {self.at_most_k_encoding.__name__}")

        while True:
            remaining_time = math.floor(timeout_timestamp - time.time())
            if remaining_time <= 0:
                print("  Overall timeout reached. Breaking optimization loop.")
                break # Overall timeout for the solve method

            self.solver.set("timeout", remaining_time * 1000) # Timeout for current check call in milliseconds
            self.solver.push() # Save state for back-tracking if current objective fails

            # Add constraint to find a strictly better solution
            if optimal_objective_value is not None:
                self.solver.add(self.total_objective < optimal_objective_value)
                print(f"  Searching for solution with total difference < {optimal_objective_value} (Remaining overall time: {remaining_time}s)...")
            else:
                print(f"  Searching for initial solution (Remaining overall time: {remaining_time}s)...")

            status = self.solver.check()
            current_elapsed_time = time.time() - init_time
            
            print(f"  Solver iteration returned: {status} (Elapsed: {current_elapsed_time:.2f}s)")

            if status == sat:
                model = self.solver.model()
                current_objective = model.evaluate(self.total_objective).as_long()
                
                print(f"    Found solution with total difference = {current_objective}")
                
                if optimal_objective_value is None or current_objective < optimal_objective_value:
                    optimal_objective_value = current_objective
                    print(f"    New best total difference found: {optimal_objective_value}")

                    best_solution_schedule = []
                    for (i, j, w, p), var in self.games_vars.items():
                        if model.evaluate(var):
                            best_solution_schedule.append((i+1, j+1, w+1, p+1))
# --- Model Creation Function (adapted from the other group's `create_model`) ---
# This function will set up the core STS constraints and variables.
# It returns the solver, games_vars, home_counts, away_counts, diff_values, total_objective
# to be used in the solve method.
def create_sts_model(n, exactly_one_encoding, at_most_k_encoding):
    NUM_TEAMS = n
    NUM_WEEKS = NUM_TEAMS - 1
    NUM_PERIODS_PER_WEEK = n // 2
    
    if NUM_TEAMS % 2 != 0:
        raise ValueError(f"Error: The number of teams (n={NUM_TEAMS}) must be an even number.")

    solver_sts = Solver()

    # --- Decision Variables ---
    games_vars = {}
    for i in range(NUM_TEAMS):
        for j in range(NUM_TEAMS):
            if i != j:
                for w in range(NUM_WEEKS):
                    for p in range(NUM_PERIODS_PER_WEEK):
                        games_vars[(i, j, w, p)] = Bool(f'g_{i+1}_{j+1}_{w+1}_{p+1}')
                        
    # --- Base Constraints ---
    # 1. Every pair plays exactly once (home or away)
    for i in range(NUM_TEAMS):
        for j in range(i+1, NUM_TEAMS):
            i_j_all_possible_matches = []
            for w in range(NUM_WEEKS):
                for p in range(NUM_PERIODS_PER_WEEK):
                    i_j_all_possible_matches.append(games_vars[(i, j, w, p)])
                    i_j_all_possible_matches.append(games_vars[(j, i, w, p)])
            solver_sts.add(exactly_one_encoding(i_j_all_possible_matches, f"pair_once_{i}_{j}"))
    
            
    # 2. Every team plays exactly once a week
    for i in range(NUM_TEAMS):
        for w in range(NUM_WEEKS):
            once_week_game = []
            for j in range(NUM_TEAMS):
                if i == j: continue
                for p in range(NUM_PERIODS_PER_WEEK):
                    once_week_game.append(games_vars[(i,j,w,p)])
                    once_week_game.append(games_vars[(j,i,w,p)])
            solver_sts.add(exactly_one_encoding(once_week_game, f"team_once_week_{i}_{w}"))
            
    # 3. Every team plays at most twice in the same period over the tournament
    for i in range(NUM_TEAMS):
        for p in range(NUM_PERIODS_PER_WEEK):
            num_games_period = []
            for j in range(NUM_TEAMS):
                if i == j: continue
                for w in range(NUM_WEEKS):
                    num_games_period.append(games_vars[(i,j,w,p)])
                    num_games_period.append(games_vars[(j,i,w,p)])
            solver_sts.add(at_most_k_encoding(num_games_period, 2, f"team_at_most_2_period_{i}_{p}"))
            
    # 4. Exactly one game in each period of every week
    for w in range(NUM_WEEKS):
        for p in range(NUM_PERIODS_PER_WEEK):
            game_in_period = []
            for i in range(NUM_TEAMS):
                for j in range(i+1, NUM_TEAMS):
                    game_in_period.append(games_vars[(i,j,w,p)])
                    game_in_period.append(games_vars[(j,i,w,p)])
            solver_sts.add(exactly_one_encoding(game_in_period, f"slot_one_game_{w}_{p}"))
            
    # Symmetry Breaking 
    
    # 1. impose team that team one plays against team 2 in the first period during the first week
    if NUM_TEAMS >= 2 and NUM_WEEKS >= 1 and NUM_PERIODS_PER_WEEK >= 1:
        solver_sts.add(games_vars[(0, 1, 0, 0)])

    # --- Optimization Variables ---
    home_counts = [Int(f'H_{i+1}') for i in range(NUM_TEAMS)]
    away_counts = [Int(f'A_{i+1}') for i in range(NUM_TEAMS)]
    diff_values = [Int(f'D_{i+1}') for i in range(NUM_TEAMS)] 

    # Constraints for home/away counts, using Z3's If and Sum for aggregation
    for i in range(NUM_TEAMS):
        home_games_for_team_i_bools = [games_vars[(i, j, w, p)] 
                                        for j in range(NUM_TEAMS) if i != j 
                                        for w in range(NUM_WEEKS) 
                                        for p in range(NUM_PERIODS_PER_WEEK)]
        solver_sts.add(home_counts[i] == Sum([If(v, 1, 0) for v in home_games_for_team_i_bools]))

        away_games_for_team_i_bools = [games_vars[(j, i, w, p)] 
                                        for j in range(NUM_TEAMS) if i != j 
                                        for w in range(NUM_WEEKS) 
                                        for p in range(NUM_PERIODS_PER_WEEK)]
        solver_sts.add(away_counts[i] == Sum([If(v, 1, 0) for v in away_games_for_team_i_bools]))

        solver_sts.add(home_counts[i] + away_counts[i] == NUM_WEEKS) 
        
        # Vincola diff_values[i] ad essere esattamente il valore assoluto
        solver_sts.add(diff_values[i] == If(home_counts[i] >= away_counts[i],
                                       home_counts[i] - away_counts[i],
                                       away_counts[i] - home_counts[i]))

    # Total difference (objective function)
    total_objective = Sum(diff_values)
    
    # --- ADDING THE LOWER BOUND CONSTRAINT HERE ---
    # Since n is always even, NUM_WEEKS (n-1) is always odd.
    # The minimum difference |H-A| for a single team is 1.
    # So, the sum of differences for all N_TEAMS is at least N_TEAMS * 1.
    lower_bound_total_diff = NUM_TEAMS # This is 'n' in your problem context
    print(f"  Adding lower bound constraint: total_objective >= {lower_bound_total_diff}")
    solver_sts.add(total_objective >= lower_bound_total_diff)

    return solver_sts, games_vars, home_counts, away_counts, diff_values, total_objective


# --- Unified Model Class (adapted from the other group's `Unified_HeuleEnc_Model`) ---
class STS_Optimized_Model:
    def __init__(self, n, exactly_one_encoding, at_most_k_encoding):
        self.n = n
        self.NUM_TEAMS = n
        self.NUM_WEEKS = self.NUM_TEAMS - 1
        self.NUM_PERIODS_PER_WEEK = self.n // 2
        self.exactly_one_encoding = exactly_one_encoding
        self.at_most_k_encoding = at_most_k_encoding
        
        # in this way i can change the encodings as i prefer
        self.solver, self.games_vars, self.home_counts, self.away_counts, self.diff_values, self.total_objective = create_sts_model(n, self.exactly_one_encoding, self.at_most_k_encoding)

    def solve(self, timeout_seconds, random_seed=None):
        set_option("sat.local_search", True) # As in their code
        if random_seed is not None:
            self.solver.set("random_seed", random_seed)
            
        #self.solver.set("phase_selection", 0) # 0: Preferenza per polarità binarie. Prova anche 1 o 2.

        #self.solver.set("restart_factor", 1.2)
        #self.solver.set("restart_strategy", 1) # 0: GEOMETRIC, 1: LUBY.

        # Init for iterative optimization
        optimal_objective_value = None
        best_solution_schedule = []
        init_time = time.time()
        timeout_timestamp = init_time + timeout_seconds

        print(f"\n--- Starting Optimization for n={self.n} ---")
        print(f"  Using exactly_one: {self.exactly_one_encoding.__name__}, at_most_k: {self.at_most_k_encoding.__name__}")

        while True:
            remaining_time = math.floor(timeout_timestamp - time.time())
            if remaining_time <= 0:
                print("  Overall timeout reached. Breaking optimization loop.")
                break # Overall timeout for the solve method

            self.solver.set("timeout", remaining_time * 1000) # Timeout for current check call in milliseconds
            self.solver.push() # Save state for back-tracking if current objective fails

            # Add constraint to find a strictly better solution
            if optimal_objective_value is not None:
                self.solver.add(self.total_objective < optimal_objective_value)
                print(f"  Searching for solution with total difference < {optimal_objective_value} (Remaining overall time: {remaining_time}s)...")
            else:
                print(f"  Searching for initial solution (Remaining overall time: {remaining_time}s)...")

            status = self.solver.check()
            current_elapsed_time = time.time() - init_time
            
            print(f"  Solver iteration returned: {status} (Elapsed: {current_elapsed_time:.2f}s)")

            if status == sat:
                model = self.solver.model()
                current_objective = model.evaluate(self.total_objective).as_long()
                
                print(f"    Found solution with total difference = {current_objective}")
                
                if optimal_objective_value is None or current_objective < optimal_objective_value:
                    optimal_objective_value = current_objective
                    print(f"    New best total difference found: {optimal_objective_value}")

                    best_solution_schedule = []
                    for (i, j, w, p), var in self.games_vars.items():
                        if model.evaluate(var):
                            best_solution_schedule.append((i+1, j+1, w+1, p+1))
# --- Model Creation Function (adapted from the other group's `create_model`) ---
# This function will set up the core STS constraints and variables.
# It returns the solver, games_vars, home_counts, away_counts, diff_values, total_objective
# to be used in the solve method.
def create_sts_model(n, exactly_one_encoding, at_most_k_encoding):
    NUM_TEAMS = n
    NUM_WEEKS = NUM_TEAMS - 1
    NUM_PERIODS_PER_WEEK = n // 2
    
    if NUM_TEAMS % 2 != 0:
        raise ValueError(f"Error: The number of teams (n={NUM_TEAMS}) must be an even number.")

    solver_sts = Solver()

    # --- Decision Variables ---
    games_vars = {}
    for i in range(NUM_TEAMS):
        for j in range(NUM_TEAMS):
            if i != j:
                for w in range(NUM_WEEKS):
                    for p in range(NUM_PERIODS_PER_WEEK):
                        games_vars[(i, j, w, p)] = Bool(f'g_{i+1}_{j+1}_{w+1}_{p+1}')
                        
    # --- Base Constraints ---
    # 1. Every pair plays exactly once (home or away)
    for i in range(NUM_TEAMS):
        for j in range(i+1, NUM_TEAMS):
            i_j_all_possible_matches = []
            for w in range(NUM_WEEKS):
                for p in range(NUM_PERIODS_PER_WEEK):
                    i_j_all_possible_matches.append(games_vars[(i, j, w, p)])
                    i_j_all_possible_matches.append(games_vars[(j, i, w, p)])
            solver_sts.add(exactly_one_encoding(i_j_all_possible_matches, f"pair_once_{i}_{j}"))
    
            
    # 2. Every team plays exactly once a week
    for i in range(NUM_TEAMS):
        for w in range(NUM_WEEKS):
            once_week_game = []
            for j in range(NUM_TEAMS):
                if i == j: continue
                for p in range(NUM_PERIODS_PER_WEEK):
                    once_week_game.append(games_vars[(i,j,w,p)])
                    once_week_game.append(games_vars[(j,i,w,p)])
            solver_sts.add(exactly_one_encoding(once_week_game, f"team_once_week_{i}_{w}"))
            
    # 3. Every team plays at most twice in the same period over the tournament
    for i in range(NUM_TEAMS):
        for p in range(NUM_PERIODS_PER_WEEK):
            num_games_period = []
            for j in range(NUM_TEAMS):
                if i == j: continue
                for w in range(NUM_WEEKS):
                    num_games_period.append(games_vars[(i,j,w,p)])
                    num_games_period.append(games_vars[(j,i,w,p)])
            solver_sts.add(at_most_k_encoding(num_games_period, 2, f"team_at_most_2_period_{i}_{p}"))
            
    # 4. Exactly one game in each period of every week
    for w in range(NUM_WEEKS):
        for p in range(NUM_PERIODS_PER_WEEK):
            game_in_period = []
            for i in range(NUM_TEAMS):
                for j in range(i+1, NUM_TEAMS):
                    game_in_period.append(games_vars[(i,j,w,p)])
                    game_in_period.append(games_vars[(j,i,w,p)])
            solver_sts.add(exactly_one_encoding(game_in_period, f"slot_one_game_{w}_{p}"))
            
    # Symmetry Breaking 
    
    # 1. impose team that team one plays against team 2 in the first period during the first week
    if NUM_TEAMS >= 2 and NUM_WEEKS >= 1 and NUM_PERIODS_PER_WEEK >= 1:
        solver_sts.add(games_vars[(0, 1, 0, 0)])

    # --- Optimization Variables ---
    home_counts = [Int(f'H_{i+1}') for i in range(NUM_TEAMS)]
    away_counts = [Int(f'A_{i+1}') for i in range(NUM_TEAMS)]
    diff_values = [Int(f'D_{i+1}') for i in range(NUM_TEAMS)] 

    # Constraints for home/away counts, using Z3's If and Sum for aggregation
    for i in range(NUM_TEAMS):
        home_games_for_team_i_bools = [games_vars[(i, j, w, p)] 
                                        for j in range(NUM_TEAMS) if i != j 
                                        for w in range(NUM_WEEKS) 
                                        for p in range(NUM_PERIODS_PER_WEEK)]
        solver_sts.add(home_counts[i] == Sum([If(v, 1, 0) for v in home_games_for_team_i_bools]))

        away_games_for_team_i_bools = [games_vars[(j, i, w, p)] 
                                        for j in range(NUM_TEAMS) if i != j 
                                        for w in range(NUM_WEEKS) 
                                        for p in range(NUM_PERIODS_PER_WEEK)]
        solver_sts.add(away_counts[i] == Sum([If(v, 1, 0) for v in away_games_for_team_i_bools]))

        solver_sts.add(home_counts[i] + away_counts[i] == NUM_WEEKS) 
        
        # Vincola diff_values[i] ad essere esattamente il valore assoluto
        solver_sts.add(diff_values[i] == If(home_counts[i] >= away_counts[i],
                                       home_counts[i] - away_counts[i],
                                       away_counts[i] - home_counts[i]))

    # Total difference (objective function)
    total_objective = Sum(diff_values)
    
    # --- ADDING THE LOWER BOUND CONSTRAINT HERE ---
    # Since n is always even, NUM_WEEKS (n-1) is always odd.
    # The minimum difference |H-A| for a single team is 1.
    # So, the sum of differences for all N_TEAMS is at least N_TEAMS * 1.
    lower_bound_total_diff = NUM_TEAMS # This is 'n' in your problem context
    print(f"  Adding lower bound constraint: total_objective >= {lower_bound_total_diff}")
    solver_sts.add(total_objective >= lower_bound_total_diff)

    return solver_sts, games_vars, home_counts, away_counts, diff_values, total_objective


# --- Unified Model Class (adapted from the other group's `Unified_HeuleEnc_Model`) ---
class STS_Optimized_Model:
    def __init__(self, n, exactly_one_encoding, at_most_k_encoding):
        self.n = n
        self.NUM_TEAMS = n
        self.NUM_WEEKS = self.NUM_TEAMS - 1
        self.NUM_PERIODS_PER_WEEK = self.n // 2
        self.exactly_one_encoding = exactly_one_encoding
        self.at_most_k_encoding = at_most_k_encoding
        
        # in this way i can change the encodings as i prefer
        self.solver, self.games_vars, self.home_counts, self.away_counts, self.diff_values, self.total_objective = create_sts_model(n, self.exactly_one_encoding, self.at_most_k_encoding)

    def solve(self, timeout_seconds, random_seed=None):
        set_option("sat.local_search", True) # As in their code
        if random_seed is not None:
            self.solver.set("random_seed", random_seed)
            
        #self.solver.set("phase_selection", 0) # 0: Preferenza per polarità binarie. Prova anche 1 o 2.

        #self.solver.set("restart_factor", 1.2)
        #self.solver.set("restart_strategy", 1) # 0: GEOMETRIC, 1: LUBY.

        # Init for iterative optimization
        optimal_objective_value = None
        best_solution_schedule = []
        init_time = time.time()
        timeout_timestamp = init_time + timeout_seconds

        print(f"\n--- Starting Optimization for n={self.n} ---")
        print(f"  Using exactly_one: {self.exactly_one_encoding.__name__}, at_most_k: {self.at_most_k_encoding.__name__}")

        while True:
            remaining_time = math.floor(timeout_timestamp - time.time())
            if remaining_time <= 0:
                print("  Overall timeout reached. Breaking optimization loop.")
                break # Overall timeout for the solve method

            self.solver.set("timeout", remaining_time * 1000) # Timeout for current check call in milliseconds
            self.solver.push() # Save state for back-tracking if current objective fails

            # Add constraint to find a strictly better solution
            if optimal_objective_value is not None:
                self.solver.add(self.total_objective < optimal_objective_value)
                print(f"  Searching for solution with total difference < {optimal_objective_value} (Remaining overall time: {remaining_time}s)...")
            else:
                print(f"  Searching for initial solution (Remaining overall time: {remaining_time}s)...")

            status = self.solver.check()
            current_elapsed_time = time.time() - init_time
            
            print(f"  Solver iteration returned: {status} (Elapsed: {current_elapsed_time:.2f}s)")

            if status == sat:
                model = self.solver.model()
                current_objective = model.evaluate(self.total_objective).as_long()
                
                print(f"    Found solution with total difference = {current_objective}")
                
                if optimal_objective_value is None or current_objective < optimal_objective_value:
                    optimal_objective_value = current_objective
                    print(f"    New best total difference found: {optimal_objective_value}")

                    best_solution_schedule = []
                    for (i, j, w, p), var in self.games_vars.items():
                        if model.evaluate(var):
                            best_solution_schedule.append((i+1, j+1, w+1, p+1))
# --- Model Creation Function (adapted from the other group's `create_model`) ---
# This function will set up the core STS constraints and variables.
# It returns the solver, games_vars, home_counts, away_counts, diff_values, total_objective
# to be used in the solve method.
def create_sts_model(n, exactly_one_encoding, at_most_k_encoding):
    NUM_TEAMS = n
    NUM_WEEKS = NUM_TEAMS - 1
    NUM_PERIODS_PER_WEEK = n // 2
    
    if NUM_TEAMS % 2 != 0:
        raise ValueError(f"Error: The number of teams (n={NUM_TEAMS}) must be an even number.")

    solver_sts = Solver()

    # --- Decision Variables ---
    games_vars = {}
    for i in range(NUM_TEAMS):
        for j in range(NUM_TEAMS):
            if i != j:
                for w in range(NUM_WEEKS):
                    for p in range(NUM_PERIODS_PER_WEEK):
                        games_vars[(i, j, w, p)] = Bool(f'g_{i+1}_{j+1}_{w+1}_{p+1}')
                        
    # --- Base Constraints ---
    # 1. Every pair plays exactly once (home or away)
    for i in range(NUM_TEAMS):
        for j in range(i+1, NUM_TEAMS):
            i_j_all_possible_matches = []
            for w in range(NUM_WEEKS):
                for p in range(NUM_PERIODS_PER_WEEK):
                    i_j_all_possible_matches.append(games_vars[(i, j, w, p)])
                    i_j_all_possible_matches.append(games_vars[(j, i, w, p)])
            solver_sts.add(exactly_one_encoding(i_j_all_possible_matches, f"pair_once_{i}_{j}"))
    
            
    # 2. Every team plays exactly once a week
    for i in range(NUM_TEAMS):
        for w in range(NUM_WEEKS):
            once_week_game = []
            for j in range(NUM_TEAMS):
                if i == j: continue
                for p in range(NUM_PERIODS_PER_WEEK):
                    once_week_game.append(games_vars[(i,j,w,p)])
                    once_week_game.append(games_vars[(j,i,w,p)])
            solver_sts.add(exactly_one_encoding(once_week_game, f"team_once_week_{i}_{w}"))
            
    # 3. Every team plays at most twice in the same period over the tournament
    for i in range(NUM_TEAMS):
        for p in range(NUM_PERIODS_PER_WEEK):
            num_games_period = []
            for j in range(NUM_TEAMS):
                if i == j: continue
                for w in range(NUM_WEEKS):
                    num_games_period.append(games_vars[(i,j,w,p)])
                    num_games_period.append(games_vars[(j,i,w,p)])
            solver_sts.add(at_most_k_encoding(num_games_period, 2, f"team_at_most_2_period_{i}_{p}"))
            
    # 4. Exactly one game in each period of every week
    for w in range(NUM_WEEKS):
        for p in range(NUM_PERIODS_PER_WEEK):
            game_in_period = []
            for i in range(NUM_TEAMS):
                for j in range(i+1, NUM_TEAMS):
                    game_in_period.append(games_vars[(i,j,w,p)])
                    game_in_period.append(games_vars[(j,i,w,p)])
            solver_sts.add(exactly_one_encoding(game_in_period, f"slot_one_game_{w}_{p}"))
            
    # Symmetry Breaking 
    
    # 1. impose team that team one plays against team 2 in the first period during the first week
    if NUM_TEAMS >= 2 and NUM_WEEKS >= 1 and NUM_PERIODS_PER_WEEK >= 1:
        solver_sts.add(games_vars[(0, 1, 0, 0)])

    # --- Optimization Variables ---
    home_counts = [Int(f'H_{i+1}') for i in range(NUM_TEAMS)]
    away_counts = [Int(f'A_{i+1}') for i in range(NUM_TEAMS)]
    diff_values = [Int(f'D_{i+1}') for i in range(NUM_TEAMS)] 

    # Constraints for home/away counts, using Z3's If and Sum for aggregation
    for i in range(NUM_TEAMS):
        home_games_for_team_i_bools = [games_vars[(i, j, w, p)] 
                                        for j in range(NUM_TEAMS) if i != j 
                                        for w in range(NUM_WEEKS) 
                                        for p in range(NUM_PERIODS_PER_WEEK)]
        solver_sts.add(home_counts[i] == Sum([If(v, 1, 0) for v in home_games_for_team_i_bools]))

        away_games_for_team_i_bools = [games_vars[(j, i, w, p)] 
                                        for j in range(NUM_TEAMS) if i != j 
                                        for w in range(NUM_WEEKS) 
                                        for p in range(NUM_PERIODS_PER_WEEK)]
        solver_sts.add(away_counts[i] == Sum([If(v, 1, 0) for v in away_games_for_team_i_bools]))

        solver_sts.add(home_counts[i] + away_counts[i] == NUM_WEEKS) 
        
        # Vincola diff_values[i] ad essere esattamente il valore assoluto
        solver_sts.add(diff_values[i] == If(home_counts[i] >= away_counts[i],
                                       home_counts[i] - away_counts[i],
                                       away_counts[i] - home_counts[i]))

    # Total difference (objective function)
    total_objective = Sum(diff_values)
    
    # --- ADDING THE LOWER BOUND CONSTRAINT HERE ---
    # Since n is always even, NUM_WEEKS (n-1) is always odd.
    # The minimum difference |H-A| for a single team is 1.
    # So, the sum of differences for all N_TEAMS is at least N_TEAMS * 1.
    lower_bound_total_diff = NUM_TEAMS # This is 'n' in your problem context
    print(f"  Adding lower bound constraint: total_objective >= {lower_bound_total_diff}")
    solver_sts.add(total_objective >= lower_bound_total_diff)

    return solver_sts, games_vars, home_counts, away_counts, diff_values, total_objective


# --- Unified Model Class (adapted from the other group's `Unified_HeuleEnc_Model`) ---
class STS_Optimized_Model:
    def __init__(self, n, exactly_one_encoding, at_most_k_encoding):
        self.n = n
        self.NUM_TEAMS = n
        self.NUM_WEEKS = self.NUM_TEAMS - 1
        self.NUM_PERIODS_PER_WEEK = self.n // 2
        self.exactly_one_encoding = exactly_one_encoding
        self.at_most_k_encoding = at_most_k_encoding
        
        # in this way i can change the encodings as i prefer
        self.solver, self.games_vars, self.home_counts, self.away_counts, self.diff_values, self.total_objective = create_sts_model(n, self.exactly_one_encoding, self.at_most_k_encoding)

    def solve(self, timeout_seconds, random_seed=None):
        set_option("sat.local_search", True) # As in their code
        if random_seed is not None:
            self.solver.set("random_seed", random_seed)
            
        #self.solver.set("phase_selection", 0) # 0: Preferenza per polarità binarie. Prova anche 1 o 2.

        #self.solver.set("restart_factor", 1.2)
        #self.solver.set("restart_strategy", 1) # 0: GEOMETRIC, 1: LUBY.

        # Init for iterative optimization
        optimal_objective_value = None
        best_solution_schedule = []
        init_time = time.time()
        timeout_timestamp = init_time + timeout_seconds

        print(f"\n--- Starting Optimization for n={self.n} ---")
        print(f"  Using exactly_one: {self.exactly_one_encoding.__name__}, at_most_k: {self.at_most_k_encoding.__name__}")

        while True:
            remaining_time = math.floor(timeout_timestamp - time.time())
            if remaining_time <= 0:
                print("  Overall timeout reached. Breaking optimization loop.")
                break # Overall timeout for the solve method

            self.solver.set("timeout", remaining_time * 1000) # Timeout for current check call in milliseconds
            self.solver.push() # Save state for back-tracking if current objective fails

            # Add constraint to find a strictly better solution
            if optimal_objective_value is not None:
                self.solver.add(self.total_objective < optimal_objective_value)
                print(f"  Searching for solution with total difference < {optimal_objective_value} (Remaining overall time: {remaining_time}s)...")
            else:
                print(f"  Searching for initial solution (Remaining overall time: {remaining_time}s)...")

            status = self.solver.check()
            current_elapsed_time = time.time() - init_time
            
            print(f"  Solver iteration returned: {status} (Elapsed: {current_elapsed_time:.2f}s)")

            if status == sat:
                model = self.solver.model()
                current_objective = model.evaluate(self.total_objective).as_long()
                
                print(f"    Found solution with total difference = {current_objective}")
                
                if optimal_objective_value is None or current_objective < optimal_objective_value:
                    optimal_objective_value = current_objective
                    print(f"    New best total difference found: {optimal_objective_value}")

                    best_solution_schedule = []
                    for (i, j, w, p), var in self.games_vars.items():
                        if model.evaluate(var):
                            best_solution_schedule.append((i+1, j+1, w+1, p+1))
                    
                    print("    Home/Away counts for this solution:")
                    for i in range(self.NUM_TEAMS):
                        h_count = model.evaluate(self.home_counts[i]).as_long()
                        a_count = model.evaluate(self.away_counts[i]).as_long()
                        d_val = model.evaluate(self.diff_values[i]).as_long()
                        calculated_diff = abs(h_count - a_count)
                        print(f"      Team {i+1}: Home = {h_count}, Away = {a_count}, Diff = {d_val} (Calculated: {calculated_diff})")

                else:
                    # This case should ideally not happen if a strict inequality constraint was added
                    # and status is SAT. It might indicate a timeout on the *inner* check call
                    # that yielded a solution not strictly better than the global best.
                    print("    Found solution, but it's not strictly better than the current best. Optimization complete.")
                    self.solver.pop() # Pop the last constraint to revert
                    break # Exit loop as no better solution found

            elif status == unsat:
                print("  No better solution found. Optimization complete.")
                self.solver.pop() # Pop the last constraint that made it unsat
                break

            elif status == unknown:
                print(f"  Solver returned 'unknown' (likely timeout within iteration or unhandled theory).")
                break # Cannot guarantee optimality, so stop

            self.solver.pop() # Remove the specific objective constraint for the last iteration
            # Loop continues to search for better solution (if 'sat' was returned)

        # Collect statistics after optimization loop finishes
        # Note: Statistics are for the *last* check call. To get statistics for
        # the entire process, you'd need to aggregate them.
        # This part mimics their statistics collection, which gets stats for the LAST check.
        stat = self.solver.statistics()
        final_stats = {
            'restarts': stat.get_key_value('restarts') if 'restarts' in stat.keys() else 0,
            'max_memory': stat.get_key_value('max memory') if 'max memory' in stat.keys() else 0,
            'mk_bool_var': stat.get_key_value('mk bool var') if 'mk bool var' in stat.keys() else 0,
            'conflicts': stat.get_key_value('conflicts') if 'conflicts' in stat.keys() else 0,
            'solve_time': current_elapsed_time # Total time for optimization
        }
        
        print("\n--- Optimization Finished ---")
        print(f"  Optimal Total Difference found: {optimal_objective_value}")
        print("  Final Solver Statistics (from last check):")
        for k, v in final_stats.items():
            print(f"    {k}: {v}")

        if best_solution_schedule:
            print(f"  Number of active matches in best solution: {len(best_solution_schedule)}")
            return best_solution_schedule, final_stats, model # Return the best model found
        else:
            print("  No solution found at all.")
            return None, final_stats, None

# --- Main Block for Execution ---
if __name__ == '__main__':
    # --- CHOOSE YOUR ENCODINGS HERE ---
    chosen_exactly_one_encoding = heule_exactly_one 
    chosen_at_most_k_encoding = at_most_k_seq   

    n_test = 12 # Example: for n=6 (5 weeks, 3 periods/week)
    test_timeout_seconds = 5 * 60 # 5 minutes overall timeout

    print(f"Using exactly_one: {chosen_exactly_one_encoding.__name__}, and at_most_k: {chosen_at_most_k_encoding.__name__}")
    print(f"Attempting to solve STS with Home/Away balance optimization for {n_test} teams (similar to previous year's exam style)...")
    
    # Instantiate the unified model
    sts_model = STS_Optimized_Model(n_test, chosen_exactly_one_encoding, chosen_at_most_k_encoding)

    # Solve the model
    match_list, stats, final_model = sts_model.solve(test_timeout_seconds, random_seed=42)

    if match_list:
        print_weekly_schedule(match_list, n_test)

        # Verify home/away counts for the optimal solution
        print("\nVerifying Home/Away balance for the final schedule:")
        for team_idx in range(n_test):
            h_count = 0
            a_count = 0
            for (i, j, w, p) in match_list:
                if i - 1 == team_idx: # i is home team
                    h_count += 1
                elif j - 1 == team_idx: # j is away team
                    a_count += 1
            print(f"  Team {team_idx+1}: Home Games = {h_count}, Away Games = {a_count}, Difference = {abs(h_count - a_count)}")